In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [ ]:
import uproot
import numpy as np
from tqdm import tqdm as tqdm
import matplotlib.pyplot as plt
import os
import optparse
import os.path as osp
import awkward as ak
import math
import torch_geometric
import torch
import sys
#import pandas as pd
from torch_geometric.data import Data
import glob

In [ ]:
def analyze(filename,nfile,minenfcut, output_dir):
    
    test = uproot.open(filename)['ana']['hgc']
    print("opened file name:",str(filename))
    print(test.num_entries)
    minenf_cut = minenfcut
    
    
    event = test['event'].array()
    gun_pid = test['gunparticle_id'].array()
    gun_en = test['gunparticle_energy'].array()
    gun_eta = test['gunparticle_eta'].array()
    gun_phi = test['gunparticle_phi'].array()

    clus_x = test['cluster2d_x'].array()
    clus_y = test['cluster2d_y'].array()
    clus_z = test['cluster2d_z'].array()
    clus_l = test['cluster2d_layer'].array()
    clus_en = test['cluster2d_energy'].array()
    clus_rechits = test['cluster2d_rechits'].array()
    clus_rechitseed = test['cluster2d_rechitSeed'].array()
    clus_t = test['cluster2d_t'].array()
    clus_dt = test['cluster2d_dt'].array()



    multi_eta = test['multiclus_eta'].array()
    multi_phi = test['multiclus_phi'].array()
    multi_clus2d = test['multiclus_cluster2d'].array()
    #multi_cluseed = test['multiclus_cl2dSeed'].array()
    multi_en = test['multiclus_energy'].array()
    multi_time = test['multiclus_time'].array()


    sim_idx  = test['simcluster_hits_indices'].array()
    #sim_idx = sim_idx[sim_idx > 0]
    sim_frac = test['simcluster_fractions'].array()
    #sim_frac = sim_frac[sim_idx > 0]
    simcluster_eta = test['simcluster_eta'].array()
    simcluster_pid = test['simcluster_pid'].array()
    simcluster_en = test['simcluster_energy'].array()
    #clus_rechits = awkward.fromiter(clus_rechits)
    #multi_clus2d = awkward.fromiter(multi_clus2d)

    calo_simidx = test['calopart_simClusterIndex'].array()
    rechit_energy = test['rechit_energy'].array()

    

    #rechit_time = test['rechit_time'].array()
    #sim_indices = test['simcluster_hits'].array()


    
    #print(len(multi_eta))
    nsskip = 0
    #ntrkster = []
    listoftensors = []
    for evt in tqdm(range(len(multi_eta)),desc='events processed'):
    
        for ngun in range(len(gun_eta[evt])):
                #print(gun_eta[evt][ngun],gun_phi[evt][ngun],gun_en[evt][ngun])

                sel_trk = multi_eta[evt]>0 #Selecting only tracksters in the positive endcap
                sel_simtrk = calo_simidx[evt][ngun]
                perc_simtrk = []
                max_perc_simtrk = 0
                max_perc_simtrk_idx = -99
                skip = True
                simen = -1
                mask = sim_idx[evt][sel_simtrk] > -1
                recHits_simTrack     = ak.flatten(sim_idx[evt][sel_simtrk][mask]) #rechits associated to the SimCluster
                entotsimrechit = sum((rechit_energy[evt][recHits_simTrack] * ak.flatten(sim_frac[evt][sel_simtrk][mask])))
                for ntk in range(len(multi_eta[evt][sel_trk])):
                    recHits_tracksters = ak.flatten(clus_rechits[evt][multi_clus2d[evt][sel_trk][ntk]]) #rechits associated to the trackster trk
                    recHits_intersection = np.intersect1d(recHits_tracksters, recHits_simTrack) #rechits belonging to both the SimCluster and the trackster
                    indices = np.where(np.isin(recHits_simTrack, recHits_intersection))[0] #indeces in the recHits_simTrack of the rechits belonging to the intersection
                    en = sum((rechit_energy[evt][recHits_simTrack] * ak.flatten(sim_frac[evt][sel_simtrk][mask]))[indices]) ## sim energy in the common rechits 
                    frac_simtrk = (en / multi_en[evt][sel_trk][ntk]) * 100 ## frac of trackster energy coming from sim
                    if (frac_simtrk > max_perc_simtrk) and (en/entotsimrechit > minenf_cut): ## trackster with max frac of sim_en in it && it has at least minenfcut frac of total sim energy
                        max_perc_simtrk = frac_simtrk
                        max_perc_simtrk_idx = ntk
                        simen = en
                        skip = False




                if (skip) :
                    nsskip += 1
                    print("skippping")
                    continue


                trkgunen = ak.to_numpy(gun_en[evt][ngun])
                trkguneta = ak.to_numpy(gun_eta[evt][ngun])
                trkgunphi = ak.to_numpy(gun_phi[evt][ngun])

                trkcluseta = (multi_eta[evt][sel_trk][max_perc_simtrk_idx])
                trkclusphi = (multi_phi[evt][sel_trk][max_perc_simtrk_idx])
                trkclusen = (multi_en[evt][sel_trk][max_perc_simtrk_idx])
                trkclustime = (multi_time[evt][sel_trk][max_perc_simtrk_idx])

                #### OUTPUT COLLS
                clusE = ak.to_numpy(clus_en[evt][multi_clus2d[evt][sel_trk][max_perc_simtrk_idx]])
                clusX = ak.to_numpy(clus_x[evt][multi_clus2d[evt][sel_trk][max_perc_simtrk_idx]])
                clusY = ak.to_numpy(clus_y[evt][multi_clus2d[evt][sel_trk][max_perc_simtrk_idx]])
                clusZ = ak.to_numpy(clus_z[evt][multi_clus2d[evt][sel_trk][max_perc_simtrk_idx]])
                ##clusZ[clusZ < 0] = -1 * clusZ[clusZ < 0] ## not required for these samples (2 phos in the +ve endcap)
                clusL = ak.to_numpy(clus_l[evt][multi_clus2d[evt][sel_trk][max_perc_simtrk_idx]])

                clusT = ak.to_numpy(clus_t[evt][multi_clus2d[evt][sel_trk][max_perc_simtrk_idx]])
                clusdT = ak.to_numpy(clus_dt[evt][multi_clus2d[evt][sel_trk][max_perc_simtrk_idx]])

                clus2d_feats = np.stack((clusX,clusY,clusZ,clusE,clusT,clusL)).T
                clus3d_feats = np.stack((trkcluseta,trkclusphi,trkclusen,trkclustime, min(clusL),max(clusL))).T
                gun_feats = np.stack((trkguneta,trkgunphi,trkgunen)).T


                datum = Data(clus2d_feat = torch.tensor(clus2d_feats, dtype=torch.float32),
                        clus3d_feat = torch.tensor(clus3d_feats, dtype=torch.float32),
                        gun_feat = torch.tensor(gun_feats, dtype=torch.float32))

                listoftensors.append(datum)

                
                
                
    processed_dir = output_dir #'/grid_mnt/data__data.polcms/cms/sghosh/NEWPID_DATA/PHO/' ## PHO
    import os
    import os.path as osp
    if not os.path.exists(processed_dir):
        os.makedirs(processed_dir)


    #torch.save(listoftensors, osp.join(processed_dir, 'data_{}_{}_{}_{}.pt'.format(nfile,evt,ngun,trkidx)))
    torch.save(listoftensors, osp.join(processed_dir, 'data_{}.pt'.format(nfile)))
    print("skipped events:",nsskip)
    #print("average #tracksters:",np.mean(ntrkster))
    return 0

#raw_dir="/grid_mnt/data__data.polcms/cms/tarabini/photons_forPID/step3/"
#raw_dir="/grid_mnt/data__data.polcms/cms/tarabini/pions_forPID/step3/"
#analyze(raw_dir+'step3_1.root',21)

In [ ]:
#raw_dir='/grid_mnt/data__data.polcms/cms/tarabini/GENPHOTESTPU2_noSmearing/step3_ticlv4_clue3d_CMSSW_12_4_0_pre2/'
raw_dir="/grid_mnt/data__data.polcms/cms/tarabini/photons_forPID/step3/" ### PHO
output_dir = '/grid_mnt/data__data.polcms/cms/sghosh/NEWPID_DATA/ntup_pho_frac0p8/'
fnamelist = os.listdir(raw_dir)#+'STEP3_*.root'
print(fnamelist)

for idx, file in enumerate(fnamelist):
    try:
        #print(raw_dir+file,idx)
        analyze(raw_dir+file,idx, 0.8, output_dir) ### PHO
        #analyze(raw_dir+file,idx, 0.99) ### PI
        #break
    except:
        print("file is invalid, skipping....")


In [ ]:
filename = "/grid_mnt/data__data.polcms/cms/tarabini/photons_forPID/step3/step3_31.root"
test = uproot.open(filename)['ana']['hgc']
print("opened file name:",str(filename))
print(test.num_entries)